# Padding and Stride
:label:`sec_padding`

Recall the example of a convolution in :numref:`fig_correlation`.
The input had both a height and width of 3
and the convolution kernel had both a height and width of 2,
yielding an output representation with dimension $2\times2$.
Assuming that the input shape is $n_\textrm{h}\times n_\textrm{w}$
and the convolution kernel shape is $k_\textrm{h}\times k_\textrm{w}$,
the output shape will be $(n_\textrm{h}-k_\textrm{h}+1) \times (n_\textrm{w}-k_\textrm{w}+1)$:
we can only shift the convolution kernel so far until it runs out
of pixels to apply the convolution to.

In the following we will explore a number of techniques,
including padding and strided convolutions,
that offer more control over the size of the output.
As motivation, note that since kernels generally
have width and height greater than $1$,
after applying many successive convolutions,
we tend to wind up with outputs that are
considerably smaller than our input.
If we start with a $240 \times 240$ pixel image,
ten layers of $5 \times 5$ convolutions
reduce the image to $200 \times 200$ pixels,
slicing off $30 \%$ of the image and with it
obliterating any interesting information
on the boundaries of the original image.
*Padding* is the most popular tool for handling this issue.
In other cases, we may want to reduce the dimensionality drastically,
e.g., if we find the original input resolution to be unwieldy.
*Strided convolutions* are a popular technique that can help in these instances.


In [1]:
import torch
from torch import nn

## Padding

As described above, one tricky issue when applying convolutional layers
is that we tend to lose pixels on the perimeter of our image. Consider :numref:`img_conv_reuse` that depicts the pixel utilization as a function of the convolution kernel size and the position within the image. The pixels in the corners are hardly used at all.

![Pixel utilization for convolutions of size $1 \times 1$, $2 \times 2$, and $3 \times 3$ respectively.](https://github.com/d2l-ai/d2l-pytorch-colab/blob/master/img/conv-reuse.svg?raw=1)
:label:`img_conv_reuse`

Since we typically use small kernels,
for any given convolution
we might only lose a few pixels
but this can add up as we apply
many successive convolutional layers.
One straightforward solution to this problem
is to add extra pixels of filler around the boundary of our input image,
thus increasing the effective size of the image.
Typically, we set the values of the extra pixels to zero.
In :numref:`img_conv_pad`, we pad a $3 \times 3$ input,
increasing its size to $5 \times 5$.
The corresponding output then increases to a $4 \times 4$ matrix.
The shaded portions are the first output element as well as the input and kernel tensor elements used for the output computation: $0\times0+0\times1+0\times2+0\times3=0$.

![Two-dimensional cross-correlation with padding.](https://github.com/d2l-ai/d2l-pytorch-colab/blob/master/img/conv-pad.svg?raw=1)
:label:`img_conv_pad`

In general, if we add a total of $p_\textrm{h}$ rows of padding
(roughly half on top and half on bottom)
and a total of $p_\textrm{w}$ columns of padding
(roughly half on the left and half on the right),
the output shape will be

$$(n_\textrm{h}-k_\textrm{h}+p_\textrm{h}+1)\times(n_\textrm{w}-k_\textrm{w}+p_\textrm{w}+1).$$

This means that the height and width of the output
will increase by $p_\textrm{h}$ and $p_\textrm{w}$, respectively.

In many cases, we will want to set $p_\textrm{h}=k_\textrm{h}-1$ and $p_\textrm{w}=k_\textrm{w}-1$
to give the input and output the same height and width.
This will make it easier to predict the output shape of each layer
when constructing the network.
Assuming that $k_\textrm{h}$ is odd here,
we will pad $p_\textrm{h}/2$ rows on both sides of the height.
If $k_\textrm{h}$ is even, one possibility is to
pad $\lceil p_\textrm{h}/2\rceil$ rows on the top of the input
and $\lfloor p_\textrm{h}/2\rfloor$ rows on the bottom.
We will pad both sides of the width in the same way.

CNNs commonly use convolution kernels
with odd height and width values, such as 1, 3, 5, or 7.
Choosing odd kernel sizes has the benefit
that we can preserve the dimensionality
while padding with the same number of rows on top and bottom,
and the same number of columns on left and right.

Moreover, this practice of using odd kernels
and padding to precisely preserve dimensionality
offers a clerical benefit.
For any two-dimensional tensor `X`,
when the kernel's size is odd
and the number of padding rows and columns
on all sides are the same,
thereby producing an output with the same height and width as the input,
we know that the output `Y[i, j]` is calculated
by cross-correlation of the input and convolution kernel
with the window centered on `X[i, j]`.

In the following example, we create a two-dimensional convolutional layer
with a height and width of 3
and (**apply 1 pixel of padding on all sides.**)
Given an input with a height and width of 8,
we find that the height and width of the output is also 8.


In [ ]:
# We define a helper function to calculate convolutions. It initializes the
# convolutional layer weights and performs corresponding dimensionality
# elevations and reductions on the input and output
def comp_conv2d(conv2d, X):
    # (1, 1) indicates that batch size and the number of channels are both 1
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    # Strip the first two dimensions: examples and channels
    return Y.reshape(Y.shape[2:])

# 1 row and column is padded on either side, so a total of 2 rows or columns
# are added
conv2d = nn.LazyConv2d(1, kernel_size=3, padding=1)
X = torch.rand(size=(8, 8))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

When the height and width of the convolution kernel are different,
we can make the output and input have the same height and width
by [**setting different padding numbers for height and width.**]


In [ ]:
# We use a convolution kernel with height 5 and width 3. The padding on either
# side of the height and width are 2 and 1, respectively
conv2d = nn.LazyConv2d(1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

## Stride

When computing the cross-correlation,
we start with the convolution window
at the upper-left corner of the input tensor,
and then slide it over all locations both down and to the right.
In the previous examples, we defaulted to sliding one element at a time.
However, sometimes, either for computational efficiency
or because we wish to downsample,
we move our window more than one element at a time,
skipping the intermediate locations. This is particularly useful if the convolution
kernel is large since it captures a large area of the underlying image.

We refer to the number of rows and columns traversed per slide as *stride*.
So far, we have used strides of 1, both for height and width.
Sometimes, we may want to use a larger stride.
:numref:`img_conv_stride` shows a two-dimensional cross-correlation operation
with a stride of 3 vertically and 2 horizontally.
The shaded portions are the output elements as well as the input and kernel tensor elements used for the output computation: $0\times0+0\times1+1\times2+2\times3=8$, $0\times0+6\times1+0\times2+0\times3=6$.
We can see that when the second element of the first column is generated,
the convolution window slides down three rows.
The convolution window slides two columns to the right
when the second element of the first row is generated.
When the convolution window continues to slide two columns to the right on the input,
there is no output because the input element cannot fill the window
(unless we add another column of padding).

![Cross-correlation with strides of 3 and 2 for height and width, respectively.](https://github.com/d2l-ai/d2l-pytorch-colab/blob/master/img/conv-stride.svg?raw=1)
:label:`img_conv_stride`

In general, when the stride for the height is $s_\textrm{h}$
and the stride for the width is $s_\textrm{w}$, the output shape is

$$\lfloor(n_\textrm{h}-k_\textrm{h}+p_\textrm{h}+s_\textrm{h})/s_\textrm{h}\rfloor \times \lfloor(n_\textrm{w}-k_\textrm{w}+p_\textrm{w}+s_\textrm{w})/s_\textrm{w}\rfloor.$$

If we set $p_\textrm{h}=k_\textrm{h}-1$ and $p_\textrm{w}=k_\textrm{w}-1$,
then the output shape can be simplified to
$\lfloor(n_\textrm{h}+s_\textrm{h}-1)/s_\textrm{h}\rfloor \times \lfloor(n_\textrm{w}+s_\textrm{w}-1)/s_\textrm{w}\rfloor$.
Going a step further, if the input height and width
are divisible by the strides on the height and width,
then the output shape will be $(n_\textrm{h}/s_\textrm{h}) \times (n_\textrm{w}/s_\textrm{w})$.

Below, we [**set the strides on both the height and width to 2**],
thus halving the input height and width.


In [ ]:
conv2d = nn.LazyConv2d(1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

Let's look at (**a slightly more complicated example**).


In [ ]:
conv2d = nn.LazyConv2d(1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape

torch.Size([2, 2])

## Summary and Discussion

Padding can increase the height and width of the output. This is often used to give the output the same height and width as the input to avoid undesirable shrinkage of the output. Moreover, it ensures that all pixels are used equally frequently. Typically we pick symmetric padding on both sides of the input height and width. In this case we refer to $(p_\textrm{h}, p_\textrm{w})$ padding. Most commonly we set $p_\textrm{h} = p_\textrm{w}$, in which case we simply state that we choose padding $p$.

A similar convention applies to strides. When horizontal stride $s_\textrm{h}$ and vertical stride $s_\textrm{w}$ match, we simply talk about stride $s$. The stride can reduce the resolution of the output, for example reducing the height and width of the output to only $1/n$ of the height and width of the input for $n > 1$. By default, the padding is 0 and the stride is 1.

So far all padding that we discussed simply extended images with zeros. This has significant computational benefit since it is trivial to accomplish. Moreover, operators can be engineered to take advantage of this padding implicitly without the need to allocate additional memory. At the same time, it allows CNNs to encode implicit position information within an image, simply by learning where the "whitespace" is. There are many alternatives to zero-padding. :citet:`Alsallakh.Kokhlikyan.Miglani.ea.2020` provided an extensive overview of those (albeit without a clear case for when to use nonzero paddings unless artifacts occur).


## Exercises

1. Given the final code example in this section with kernel size $(3, 5)$, padding $(0, 1)$, and stride $(3, 4)$,
   calculate the output shape to check if it is consistent with the experimental result.
1. For audio signals, what does a stride of 2 correspond to?
1. Implement mirror padding, i.e., padding where the border values are simply mirrored to extend tensors.
1. What are the computational benefits of a stride larger than 1?
1. What might be statistical benefits of a stride larger than 1?
1. How would you implement a stride of $\frac{1}{2}$? What does it correspond to? When would this be useful?


[Discussions](https://discuss.d2l.ai/t/68)


### Exercise 1:

We have:

$
n_h = n_w = 8 \\
k_h = 3, k_w = 5 \\
p_h = 0, p_w = 1 \\
s_h = 3, s_w = 4 \\
$

Hence, the shape is:

$$
\begin{align*}
& \lfloor(n_\textrm{h}-k_\textrm{h}+p_\textrm{h}+s_\textrm{h})/s_\textrm{h}\rfloor \times \lfloor(n_\textrm{w}-k_\textrm{w}+p_\textrm{w}+s_\textrm{w})/s_\textrm{w}\rfloor \\
= & \lfloor(8 - 3 + 0 + 3)/3 \rfloor \times \lfloor (8 - 5 + 1 + 4)/4 \rfloor \\
= & \ 2 \times 2
\end{align*}
$$

which is consistent with the experimental result.

### Exercise 2:

In audio signal processing, a **stride of 2** typically refers to the step size when applying a sliding window operation, such as in convolution or feature extraction.

1. **Downsampling by a Factor of 2**

   - If the stride of 2 is applied during feature extraction (e.g., in a convolutional layer), it means that every second sample is skipped, effectively reducing the temporal resolution of the signal.
   - This is equivalent to halving the sampling rate, which results in losing high-frequency components if not properly filtered.

2. **Frame Shifting in STFT or Feature Extraction**

   - In a Short-Time Fourier Transform (STFT) or Mel spectrogram calculation, a stride of 2 (or hop length of 2) means that the analysis window moves forward by 2 samples at each step.
   - This would produce a denser representation compared to a larger stride.

3. **Stride in a Convolutional Neural Network (CNN) for Audio**

   - If a CNN is processing an audio spectrogram with a convolutional kernel and a stride of 2, the feature map resolution is reduced, capturing larger-scale patterns while reducing computational cost.

### Exercise 3:

In [12]:
X = torch.arange(1, 65, 1).reshape(8, 8)

X_mirror_padded = nn.functional.pad(X.reshape((1,1,8,8)), (1,1,1,1), mode='reflect').reshape(10, 10)
X, X_mirror_padded

(tensor([[ 1,  2,  3,  4,  5,  6,  7,  8],
         [ 9, 10, 11, 12, 13, 14, 15, 16],
         [17, 18, 19, 20, 21, 22, 23, 24],
         [25, 26, 27, 28, 29, 30, 31, 32],
         [33, 34, 35, 36, 37, 38, 39, 40],
         [41, 42, 43, 44, 45, 46, 47, 48],
         [49, 50, 51, 52, 53, 54, 55, 56],
         [57, 58, 59, 60, 61, 62, 63, 64]]),
 tensor([[10,  9, 10, 11, 12, 13, 14, 15, 16, 15],
         [ 2,  1,  2,  3,  4,  5,  6,  7,  8,  7],
         [10,  9, 10, 11, 12, 13, 14, 15, 16, 15],
         [18, 17, 18, 19, 20, 21, 22, 23, 24, 23],
         [26, 25, 26, 27, 28, 29, 30, 31, 32, 31],
         [34, 33, 34, 35, 36, 37, 38, 39, 40, 39],
         [42, 41, 42, 43, 44, 45, 46, 47, 48, 47],
         [50, 49, 50, 51, 52, 53, 54, 55, 56, 55],
         [58, 57, 58, 59, 60, 61, 62, 63, 64, 63],
         [50, 49, 50, 51, 52, 53, 54, 55, 56, 55]]))

### Exercise 4:

A stride larger than 1 offers several computational benefits, particularly in audio signal processing and deep learning applications:

1. **Reduced Computational Complexity**  
   - By skipping samples or feature map positions, fewer operations (e.g., convolutions or multiplications) are needed, leading to faster processing times and lower computational cost.

2. **Lower Memory Usage**  
   - Since fewer intermediate results are stored, memory requirements decrease, making it feasible to process large audio datasets or deep neural networks with limited resources.

3. **Faster Training and Inference in Neural Networks**  
   - In convolutional neural networks (CNNs), using a larger stride reduces the size of the feature maps, leading to fewer parameters and faster forward and backward passes.

4. **Efficient Downsampling and Feature Extraction**  
   - In audio processing, a larger stride can be used to downsample the signal, preserving important features while discarding redundant information, which is useful for speech and music analysis.

### Exercise 5:

A stride larger than 1 can provide several statistical benefits in audio signal processing and machine learning:

1. **Reduction in Redundancy**  
   - Many audio signals exhibit strong temporal correlation, meaning adjacent samples contain similar information. A larger stride helps reduce redundancy, leading to more diverse and independent feature representations.

2. **Improved Generalization in Machine Learning Models**  
   - By reducing overfitting to fine-grained details, a larger stride forces the model to learn more abstract and robust features rather than memorizing noise or unnecessary high-frequency variations.

3. **Smoother Feature Representations**  
   - In spectrograms or feature extraction methods like Mel-frequency cepstral coefficients (MFCCs), a larger stride results in smoother temporal or spectral representations, which can be beneficial for downstream tasks like speech recognition.

4. **Better Noise Robustness**  
   - Since a stride larger than 1 effectively averages out fine-grained variations, it can help mitigate the impact of noise, particularly in environments where background interference is present.

### Exercise 6:

### **What is a Stride of 1/2?**  
A **stride of 1/2** means moving the window by half a step, effectively overlapping adjacent windows. Since standard convolution or pooling operations in frameworks like PyTorch do not support fractional strides directly, achieving a stride of 1/2 requires **interpolation (upsampling)** before applying the convolution or using **dilated convolutions**.

### **What Does It Correspond To?**  
A stride of 1/2 corresponds to **upsampling** the input before applying a stride-1 operation. In the time domain, this is equivalent to increasing the sample rate (e.g., from 16 kHz to 32 kHz) and then applying a filter. In feature extraction, it means **denser feature maps** with more temporal resolution.

### **When is a Stride of 1/2 Useful?**  
1. **Super-resolution of Audio Signals** – Helps reconstruct missing information when increasing sampling rates.  
2. **Denoising and Smoothing** – Aids in creating finer-grained representations.  
3. **Improving Model Performance in CNNs** – Provides finer spatial resolution, especially for tasks like speech recognition.  
4. **Overlapping Frame Analysis** – Used in STFT or Mel spectrograms for smoother feature transitions.  


In [27]:
torch.manual_seed(42)

class HalfStrideConvTranspose(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super().__init__()
        self.conv_transpose = nn.ConvTranspose1d(in_channels, out_channels, kernel_size, stride=2, padding=1, output_padding=1)

    def forward(self, X):
        return self.conv_transpose(X)

model = HalfStrideConvTranspose(1, 1, 3)
X = torch.randn(1, 1, 10)
output = model(X)
X, X.shape, output, output.shape

(tensor([[[ 0.2345,  0.2303, -1.1229, -0.1863,  2.2082, -0.6380,  0.4617,
            0.2674,  0.5349,  0.8094]]]),
 torch.Size([1, 1, 10]),
 tensor([[[ 0.6427,  0.6003,  0.6407,  0.0036, -0.0077,  0.6000,  0.4411,
            1.5303,  1.5885, -0.0499,  0.2246,  0.8204,  0.7516,  0.5859,
            0.6585,  0.7303,  0.7867,  0.8153,  0.9182,  0.4209]]],
        grad_fn=<ConvolutionBackward0>),
 torch.Size([1, 1, 20]))